In [1]:
import tensorflow as tf
import pandas as pd
from tensorflow.keras.models import load_model
import pickle
import numpy as ny

In [2]:
#load the pkl files,scaler files,trained models
model=load_model('model.h5')

#load the encoder and scaler

with open('label_encoder_geo.pkl','rb') as file:
  label_encoder_geo=pickle.load(file)

with open('onehot_encoder_geo.pkl','rb') as file:
  one_hot_encoder=pickle.load(file)

with open("label_encoder_gender.pkl",'rb') as file:
  label_encoder_gender=pickle.load(file)

with open('ssclar.pkl','rb') as file:
  scalar=pickle.load(file)

In [3]:
input_data={
  'CreditScore':600,
  'Geography':'France',
  'Gender':'Male',
  'Age':40,
  'Tenure':3,
  'Balance':60000,
  'NumofProdcuts':2,
  'HascrCard':1,
  'IsActiveMember':1,
  'EstimatedSalary':5000
}

In [4]:
from sklearn.preprocessing import OneHotEncoder
import pandas as pd

# Your input data (new sample for prediction)
input_data = {
  'CreditScore': 600,
  'Geography': 'France',
  'Gender': 'Male',
  'Age': 40,
  'Tenure': 3,
  'Balance': 60000,
  'NumofProdcuts': 2,
  'HascrCard': 1,
  'IsActiveMember': 1,
  'EstimatedSalary': 5000
}

# ---- You need your full training dataset to fit the encoder correctly ----
# Let's simulate a minimal example training data:
training_data = pd.DataFrame({
    'Geography': ['France', 'Germany', 'Spain']
})

# Step 1: Fit encoder on the training data
one_hot_encoder = OneHotEncoder(handle_unknown='ignore')
one_hot_encoder.fit(training_data[['Geography']])

# Step 2: Convert input_data to a DataFrame
input_df = pd.DataFrame([input_data])  # one-row DataFrame

# Step 3: Transform the 'Geography' field
geo_encoded = one_hot_encoder.transform(input_df[['Geography']]).toarray()

# Step 4: Convert to DataFrame with feature names
geo_encoded_df = pd.DataFrame(geo_encoded, columns=one_hot_encoder.get_feature_names_out(['Geography']))

print(geo_encoded_df)


   Geography_France  Geography_Germany  Geography_Spain
0               1.0                0.0              0.0


In [5]:
input_df = pd.DataFrame([input_data])
input_df

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumofProdcuts,HascrCard,IsActiveMember,EstimatedSalary
0,600,France,Male,40,3,60000,2,1,1,5000


In [6]:
from sklearn.preprocessing import LabelEncoder
import pandas as pd
input_df = pd.DataFrame([input_data])

# Step 2: Fit LabelEncoder on all possible Gender values
training_gender = ['Male', 'Female']
label_encoder_gender = LabelEncoder()
label_encoder_gender.fit(training_gender)

# Step 3: Encode the Gender column in input_df
input_df['Gender'] = label_encoder_gender.transform(input_df['Gender'])

print(input_df)

   CreditScore Geography  Gender  Age  Tenure  Balance  NumofProdcuts  \
0          600    France       1   40       3    60000              2   

   HascrCard  IsActiveMember  EstimatedSalary  
0          1               1             5000  


In [7]:
##concatinate onehot encoded data 
input_df=pd.concat([input_df.drop("Geography",axis=1),geo_encoded_df],axis=1)
input_df

,CreditScore,Gender,Age,Tenure,Balance,NumofProdcuts,HascrCard,IsActiveMember,EstimatedSalary,Geography_France,Geography_Germany,Geography_Spain
0,600,1,40,3,60000,2,1,1,5000,1.0,0.0,0.0


In [8]:
##scaling the data
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
scaler.fit(input_df)
input_scaled=scaler.transform(input_df)
input_scaled

array([[0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.]])

In [9]:
##precistion
my_predcition=model.predict(input_scaled)
my_predcition

1/1 [==============================] - 0s 180ms/step


array([[0.18109116]], dtype=float32)

In [10]:
prediction_proba=my_predcition[0][0]

In [11]:
prediction_proba

0.18109116

In [12]:
if prediction_proba>0.5:
  print('The customer is likely to chun')
else:
  print('The custoemr is likely to not chun')

The custoemr is likely to not chun
